In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
import tensorflow as tf

print(tf.test.is_gpu_available())

train_df = pd.read_csv('task3_2021.training.csv')
test_df = pd.read_csv('task3_2021.test.csv')
train_y = train_df['target']
train_x = train_df.drop(['target'], axis=1)
train_x

# scaler = StandardScaler()
# train_x = scaler.fit_transform(train_x)
# test_x = scaler.transform(test_df)

# mse = make_scorer(mean_squared_error, squared=False)

False


,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18
0,14.557571,11.846020,15.020207,2.623484,15.538517,2.341611,5.829028,7.586709,13.331953,8.304782,-8.980424,-1.978135,-9.056215,0.601958,-3.990212,F,F,A,-8.435632
1,17.031884,4.314536,19.694838,15.264186,7.054585,0.382333,4.650885,11.331481,12.492329,12.877221,-1.788222,-12.520625,1.183564,2.312428,-5.997316,F,A,F,13.314504
2,24.986272,6.139161,21.949963,5.590123,16.445212,2.213986,2.393948,16.885378,8.114116,20.133208,12.148553,-11.400213,15.273799,2.714855,6.574276,A,C,F,8.964849
3,7.584971,11.598161,9.793109,11.873020,6.645073,3.346882,4.445008,13.218360,8.495786,17.403693,-3.371434,-1.329931,4.307364,1.217894,-1.730330,D,D,B,13.563612
4,11.625079,12.517998,15.522108,18.269750,1.508771,3.078033,5.224333,18.335198,9.604704,19.732707,3.192207,2.195230,10.903963,2.317327,1.379398,B,C,B,-8.590211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,30.169278,6.529102,19.936471,6.940652,15.873789,2.229920,1.756983,15.596095,8.405938,17.605060,7.555225,1.654488,5.563992,2.255522,4.277612,D,F,D,-9.124652
9996,9.618699,11.692794,8.343202,12.532240,12.552316,3.515778,3.795830,7.531049,11.165724,6.706453,-7.718650,-0.554662,3.445338,0.096477,-2.494105,E,D,C,5.343793
9997,9.586230,11.847254,17.784779,9.845919,15.508049,1.126731,5.809205,7.942348,10.943044,6.116932,-9.138276,-2.458970,-2.345748,0.948709,-3.642473,E,F,A,-6.140912
9998,7.255072,10.727725,7.750731,13.569023,9.702580,3.321615,4.166782,15.657056,8.861423,21.421654,-3.027806,5.185389,7.660078,1.776680,1.070294,B,B,E,2.394185


In [ ]:
class DNN(tf.keras.Model):
  def __init__(self):
    super(DNN,self).__init__()
    self.dense1 = tf.keras.layers.Dense(30, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(30, activation=tf.nn.relu)
    self.dense3 = tf.keras.layers.Dense(30, activation=tf.nn.relu)
    self.dense4 = tf.keras.layers.Dense(30, activation=tf.nn.relu)
    self.dense5 = tf.keras.layers.Dense(30, activation=tf.nn.relu)
    self.dense6 = tf.keras.layers.Dense(30, activation=tf.nn.relu)
    self.dense7 = tf.keras.layers.Dense(30, activation=tf.nn.relu)
    self.output_layer = tf.keras.layers.Dense(1)

  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    x = self.dense3(x)
    x = self.dense4(x)
    x = self.dense5(x)
    x = self.dense6(x)
    x = self.dense7(x)
    return self.output_layer(x)

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    elif epoch < 90:
        return 1e-3
    elif epoch < 400:
        return 1e-5
    elif epoch < 900:
        return 1e-6
    elif epoch < 10000:
        return 1e-7


callback_early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=100,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

batch_size = 100
epochs = 10000
opt = tf.keras.optimizers.Adam(learning_rate=1e-2)
model = DNN()
model.compile(loss="mean_squared_error", optimizer=opt, metrics=["mean_squared_error"])
model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[callback_lr,callback_early], verbose=1)
model.summary()

In [ ]:
pred = model.predict(train_x).reshape(-1)
mean_squared_error(train_y, pred, squared=False)

In [ ]:
from pandas import RangeIndex

nn_pred = model.predict(test_x)
output = nn_pred.reshape(-1)

pd.Series(output, index=RangeIndex(1, 2001), name='Predicted').to_csv('ans_nn_3.csv',index_label='id')